In [ ]:
import pandas as pd
import numpy as np

import re
import spacy
import pickle
import scispacy
from spacy.language import Language
from spacy.tokens import Span, Doc
from spacy.matcher import PhraseMatcher
from scispacy.linking import EntityLinker
from negspacy.negation import Negex
from negspacy.termsets import termset
from spacy.util import filter_spans

from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer 
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_colwidth = 100

# Load data

In [ ]:
df = pd.read_csv("../../data/rmh_cleaned.csv")
print(df.shape)
df.head()

### Tokenize and filter

In [ ]:
@Language.component("custom_ner") 
def custom_ner(doc):
    ents = []
    for token in doc:
        if not token.is_stop and not token.is_punct and not token.like_num and token.text!="+":
            ents.append(Span(doc, token.i, token.i+1, label="ENTITY"))
    doc.ents = ents
    return doc

In [ ]:
# Load scispacy model
nlp = spacy.load("en_core_sci_lg", disable=['ner'])

# Custom NER 
nlp.add_pipe("custom_ner", last=True)

print("NLP pipeline: tokenizer + {}".format(nlp.pipe_names))

# Modify negex termsets
ts = termset('en_clinical').get_patterns()
ts['preceding_negations'].extend(["nil", "non"])
ts['termination'].extend([",", ";", ":", "obviously"])

In [ ]:
%%time
df['doc'] = df.text_clean.apply(nlp)

### Create a list of common bigrams

In [ ]:
def word_count(data):
    vectorizer = CountVectorizer(stop_words=stopwords.words('english') + ts['preceding_negations'], 
                                 ngram_range=(2,2), 
                                 token_pattern=r'\S+')
    vectors = vectorizer.fit_transform(data)
    
    vocab = list(vectorizer.get_feature_names())
    counts = vectors.sum(axis=0).A1
    
    return Counter(dict(zip(vocab, counts)))

In [ ]:
bigrams = word_count(df.doc.apply(lambda x: " ".join([ent.text for ent in x.ents])))
len(bigrams)

In [ ]:
cut_off = np.quantile(np.fromiter(bigrams.values(), dtype=np.int), 0.99) 
print("Cut-off:", cut_off)
n_bigrams = (np.fromiter(bigrams.values(), dtype=np.int) > cut_off).sum()
print("%d most common bigrams" % n_bigrams)
most_common_bigrams = [item[0] for item in bigrams.most_common(n_bigrams)]

In [ ]:
with open ("../../data/most_common_bigrams.txt", 'wb') as f:
    pickle.dump(most_common_bigrams, f)

### The rest of NLP pipeline

In [ ]:
with open ("../../data/most_common_bigrams.txt", 'rb') as f:
    most_common_bigrams = pickle.load(f)

In [ ]:
def bigram_detector(doc):
    matches = matcher(doc)
    spans = [Span(doc, start, end) for _, start, end in matches]
    filtered = filter_spans(spans)
    with doc.retokenize() as retokenizer:
        for span in filtered:
            retokenizer.merge(span)
    return doc

In [ ]:
def get_canonical_name(span):
    if span._.kb_ents:
        concept = linker.kb.cui_to_entity[span._.kb_ents[0][0]].canonical_name.lower()
        return re.sub("\W", "_", concept)
    else:
        return span.text
    
def format_merged_tokens(span):
    return re.sub("\s", "_", span.text)

def apply_transformation(span, transform=""):
    if transform == "linked":
        return span._.linked
    elif transform == "merged":
        return span._.merged
    else:
        return span.text

def add_negation(span, transform=""):
    return span._.negex * "neg_" + span._.transformed(transform)
    
def prepare_tokens(doc, negation=False, transform=""):
    if negation:
        return " ".join([ent._.negated(transform) for ent in doc.ents])
    else:
        return " ".join([ent._.transformed(transform) for ent in doc.ents])

In [ ]:
bigram_patterns = list(nlp.pipe(most_common_bigrams))
matcher = PhraseMatcher(nlp.vocab)
matcher.add("BIGRAM", None, *bigram_patterns)

# Bigram detector
nlp.add_pipe(bigram_detector, last=True)

# Entity linker
linker = EntityLinker(name="mesh", threshold=0.9)
nlp.add_pipe(linker, last=True)

# Negation detector
nlp.add_pipe("negex", config={'ent_types': ['ENTITY'], 
                              'neg_termset':{
            preceding_negations": ["not"],
            "following_negations":["declined"],
            "termination": ["but","however"]
        }
    }
    )

print("NLP pipeline: tokenizer + {}".format(nlp.pipe_names))

Span.set_extension("linked", getter=get_canonical_name, force=True)
Span.set_extension("merged", getter=format_merged_tokens, force=True)
Span.set_extension("transformed", method=apply_transformation, force=True)
Span.set_extension("negated", method=add_negation, force=True)
Doc.set_extension("entities", method=prepare_tokens, force=True)

**Entities and negated entities**

In [ ]:
%%time
with nlp.disable_pipes(["bigram_detector", "EntityLinker"]):
    df['doc'] = df.text_clean.apply(nlp)

In [ ]:
df['entities'] = df.doc.apply(lambda x: x._.entities())
df['neg_entities'] = df.doc.apply(lambda x: x._.entities(negation=True))

In [ ]:
df.drop(columns="doc").to_csv("../../data/rmh_prepared_ents.csv", index=False)

**Merged entities and negated merged entities**

In [ ]:
%%time
with nlp.disable_pipes(["EntityLinker"]):
    df['doc'] = df.text_clean.apply(nlp)

In [ ]:
df['merged_entities'] = df.doc.apply(lambda x: x._.entities(transform="merged"))
df['neg_merged_entities'] = df.doc.apply(lambda x: x._.entities(negation=True, transform="merged"))

In [ ]:
df.drop(columns="doc").to_csv("../../data/rmh_prepared_merged.csv", index=False)

**Linked entities and negated linked entities**

In [ ]:
%%time
with nlp.disable_pipes(["bigram_detector"]):
    df['doc'] = df.text_clean.apply(nlp)

In [ ]:
df['linked_entities'] = df.doc.apply(lambda x: x._.entities(transform="linked"))
df['neg_linked_entities'] = df.doc.apply(lambda x: x._.entities(negation=True, transform="linked"))

In [ ]:
df.drop(columns="doc").to_csv("../../data/rmh_prepared_linked.csv", index=False)

### Merge datasets

In [ ]:
df1 = pd.read_csv("../../data/rmh_prepared_ents.csv")
df1.shape

In [ ]:
df2 = pd.read_csv("../../data/rmh_prepared_merged.csv")
df2.shape

In [ ]:
df = df1.merge(df2[["merged_entities", "neg_merged_entities"]], left_index=True, right_index=True)

In [ ]:
df3 = pd.read_csv("../../data/rmh_prepared_linked.csv")
df3.shape

In [ ]:
df = df.merge(df3[["linked_entities", "neg_linked_entities"]], left_index=True, right_index=True)

In [ ]:
df.to_csv("../../data/rmh_prepared.csv", index=False)